In [35]:
#importing necessary libraries
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [36]:
# a screenshot of the arena image is loaded

image = cv2.imread('/home/deepakachu/Desktop/eyantra_stage_2/experimetation/arena.png',cv2.IMREAD_COLOR)

In [37]:
# the image is converted to grayscale 

image_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [38]:
# a black mask is made
# an array will store the coordinates of the bottom left corner of the images

coordinates = []
mask = np.zeros_like(image_gray)

# the threshold is set. Here we choose the target threshold to be close to white and the max value to be white (since the borders of the disaster 
# images are white)
    
thresh, image_edges = cv2.threshold(image_gray,254,255,cv2.THRESH_BINARY)

# the contours are found and sorted in a contours array. For each contour, if the width and height are the same and the
# ratio of the arena width to the image width falls in range, then we add it to the mask in colour 
# using the bitwise AND 

contours,_ = cv2.findContours(image_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours,key=cv2.contourArea)
for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    if w/h == 1 and image.shape[0] / w >= 10 and image.shape[0] / w <= 14:
        cv2.drawContours(mask, [contour],-1, 255, -1)
        result = cv2.bitwise_and(image, image, mask=mask)
        coordinates.append([x,y,w,h])


In [39]:
# cv2.imshow("image",result)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [40]:
coordinates

[[150, 593, 65, 65],
 [153, 115, 65, 65],
 [460, 331, 65, 65],
 [141, 330, 65, 65]]

In [41]:
image_set = []

# all the images are put in an image array and resized with white borders removed

for coordinate in coordinates:
    x = coordinate[0] + 5
    y = coordinate[1] + 5
    w = coordinate[2] - 10
    h = coordinate[3] - 10
    img = cv2.resize(result[y:y+h, x:x+w],(50,50), interpolation = cv2.INTER_LINEAR)
    image_set.append(img)

In [ ]:
for img in image_set:
    cv2.imshow('image',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

QApplication: invalid style override 'kvantum' passed, ignoring it.
	Available styles: Windows, Fusion


In [ ]:
# images are saved in the current working directory

i = 1
for img in image_set:
    cv2.imwrite(f"img{i}.png",img)
    i = i + 1

In [ ]:
import tensorflow as tf

In [ ]:
# loading the model for image classification (the outputs shown below is NOT corresponding
# to the model that will be used finally)

model = tf.keras.models.load_model('/home/deepakachu/Desktop/eyantra_stage_2/task2b/saved_models/model_2b_revamped_test')

In [ ]:
# based on softmax output, assigning classes predicted by the model

predictions = []
for img in image_set:
    pred_arr = model.predict(np.array([img]))
    pred_val = np.where(pred_arr[0] == max(pred_arr[0]))[0][0]
    if pred_val == 0:
        pred = "combat"
    elif pred_val == 1:
        pred = 'human_aid_rehabilitation'
    elif pred_val == 2:
        pred = 'fire'
    elif pred_val == 3:
        pred = 'military_vehicles'
    elif pred_val == 4:
        pred = 'destroyed_buildings'
    predictions.append(pred)

In [ ]:
predictions

In [ ]:
# adding the bounding box (green) with class description (green) to the image
# coordinates are stored in the coordinates array (done earlier)

for i,coordinate in enumerate(coordinates):
    x = coordinate[0] 
    y = coordinate[1] 
    w = coordinate[2] 
    h = coordinate[3]
    img_with_box = cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0), 2)
    cv2.putText(image, predictions[i], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

In [ ]:
cv2.imshow("imagebox",img_with_box)
cv2.waitKey(0)
cv2.destroyAllWindows()